<a href="https://colab.research.google.com/github/Sheikhatiff/pdc-credit-card-pattern-matching/blob/main/Project_PDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PDC PROJECT :**
# ***`CREDIT CARD PATTERN MATCHING`***

## ***CREATED BY***


*   *MUHAMMAD ATIF SHEIKH    - SP22-BCS-010-8A*
*   *M ABDULLAH JAWED - SP22-BCS-032-8A*


using cuda + openMP...

## **Headers and Constants**

In [29]:
%%writefile pdc-project-sp22-bcs-010-and-032.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <omp.h>
#include <cuda_runtime.h>

// Card type constants
#define VISA 0
#define MASTERCARD 1
#define CARD_LENGTH 16
#define NUM_CARDS 10000000

// Card structure
typedef struct {
    long long number;
    int type;
} CreditCard;

Overwriting pdc-project-sp22-bcs-010-and-032.cu


## **Methods to Generate cards**

*   Generate random digit 0-9
*   Generate Visa card number (starts with 4)
*   Generate Mastercard number (starts with 51-55)  



In [30]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
// Generate random digit 0-9
int generate_random_digit() {
    return rand() % 10;
}

// Generate Visa card number (starts with 4)
long long generate_visa_number() {
    long long number = 4;
    for (int i = 1; i < CARD_LENGTH; i++) {
        number = number * 10 + generate_random_digit();
    }
    return number;
}

// Generate Mastercard number (starts with 51-55)
long long generate_mastercard_number() {
    long long number = 51 + (rand() % 5);
    for (int i = 2; i < CARD_LENGTH; i++) {
        number = number * 10 + generate_random_digit();
    }
    return number;
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **Initialize cards using OpenMP**

In [31]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
void initialize_cards_parallel(CreditCard* cards, int count) {
    srand(time(NULL));
    #pragma omp parallel for
    for (int i = 0; i < count; i++) {
        unsigned int seed = time(NULL) + omp_get_thread_num() + i;
        int local_rand = rand_r(&seed);
        cards[i].type = local_rand % 2;
        if (cards[i].type == VISA) {
            cards[i].number = generate_visa_number();
        } else {
            cards[i].number = generate_mastercard_number();
        }
    }
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **GPU: Check card pattern**

In [32]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
__device__ int check_card_pattern_gpu(long long number, int type) {
    long long first_two = number;
    while (first_two >= 100) {
        first_two /= 10;
    }
    long long first_digit = first_two / 10;

    if (type == VISA) {
        return (first_digit == 4) ? 1 : 0;
    } else {
        return (first_two >= 51 && first_two <= 55) ? 1 : 0;
    }
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **GPU: Luhn algorithm**

In [33]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
__device__ int luhn_check_gpu(long long number) {
    int sum = 0;
    int alternate = 0;

    while (number > 0) {
        int digit = number % 10;
        if (alternate) {
            digit *= 2;
            if (digit > 9) {
                digit = (digit % 10) + (digit / 10);
            }
        }
        sum += digit;
        number /= 10;
        alternate = !alternate;
    }

    return (sum % 10 == 0) ? 1 : 0;
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **GPU: Validate cards kernel**

In [34]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
__global__ void validate_cards_kernel(CreditCard* cards, int* results, int count) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < count) {
        long long number = cards[idx].number;
        int type = cards[idx].type;
        int pattern_valid = check_card_pattern_gpu(number, type);
        int luhn_valid = luhn_check_gpu(number);
        results[idx] = pattern_valid && luhn_valid;
    }
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **CPU: Count valid cards using OpenMP**

In [35]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
int count_valid_cards_parallel(int* results, int count) {
    int valid_count = 0;
    #pragma omp parallel for reduction(+:valid_count)
    for (int i = 0; i < count; i++) {
        valid_count += results[i];
    }
    return valid_count;
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **Hybrid validation pipeline**

In [36]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
int validate_cards_hybrid(CreditCard* cards, int count, double* cuda_time) {
    CreditCard* d_cards;
    int* d_results;
    int* h_results;

    h_results = (int*)malloc(count * sizeof(int));
    cudaMalloc(&d_cards, count * sizeof(CreditCard));
    cudaMalloc(&d_results, count * sizeof(int));

    cudaMemcpy(d_cards, cards, count * sizeof(CreditCard), cudaMemcpyHostToDevice);

    double cuda_start = omp_get_wtime();
    int threads_per_block = 512;
    int num_blocks = (count + threads_per_block - 1) / threads_per_block;
    validate_cards_kernel<<<num_blocks, threads_per_block>>>(d_cards, d_results, count);
    cudaDeviceSynchronize();
    double cuda_end = omp_get_wtime();
    *cuda_time = cuda_end - cuda_start;

    cudaMemcpy(h_results, d_results, count * sizeof(int), cudaMemcpyDeviceToHost);

    int valid_count = count_valid_cards_parallel(h_results, count);

    cudaFree(d_cards);
    cudaFree(d_results);
    free(h_results);

    return valid_count;
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **Save sample cards to JSON file**

In [37]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
void save_sample_cards(CreditCard* cards, int* results, int count, int sample_size) {
    FILE* fp = fopen("sample_cards.json", "w");
    fprintf(fp, "[\n");

    int saved = 0;
    for (int i = 0; i < count && saved < sample_size; i++) {
        if (saved > 0) fprintf(fp, ",\n");
        fprintf(fp, "  {\"id\": %d, \"number\": \"%lld\", \"type\": \"%s\", \"valid\": %s}",
                i + 1, cards[i].number,
                cards[i].type == VISA ? "VISA" : "MASTERCARD",
                results[i] ? "true" : "false");
        saved++;
    }

    fprintf(fp, "\n]\n");
    fclose(fp);
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **Main Function of project**

*   Generate cards
*   Validate using CUDA
*   Calculate metrics
*   Save results to JSON
*   Save sample cards (first 1000)
*   Re-validate to get individual results for sampling
*   Display Summary



In [38]:
%%writefile -a pdc-project-sp22-bcs-010-and-032.cu
int main() {
    CreditCard* cards = (CreditCard*)malloc(NUM_CARDS * sizeof(CreditCard));

    // Phase 1: Generate cards
    printf("Generating %d credit cards using OpenMP...\n", NUM_CARDS);
    double gen_start = omp_get_wtime();
    initialize_cards_parallel(cards, NUM_CARDS);
    double gen_end = omp_get_wtime();
    double gen_time = gen_end - gen_start;
    printf("Generation complete: %.4f seconds\n\n", gen_time);

    // Phase 2: Validate using CUDA
    printf("Validating cards using CUDA + OpenMP...\n");
    double cuda_time = 0.0;
    double val_start = omp_get_wtime();
    int valid_count = validate_cards_hybrid(cards, NUM_CARDS, &cuda_time);
    double val_end = omp_get_wtime();
    double val_time = val_end - val_start;
    printf("Validation complete: %.4f seconds\n\n", val_time);

    // Calculate metrics
    double total_time = gen_time + val_time;
    double throughput = NUM_CARDS / total_time;
    int invalid_count = NUM_CARDS - valid_count;
    double valid_percentage = (valid_count * 100.0) / NUM_CARDS;

    // Save results to JSON
    printf("💾 Saving results to JSON...\n");
    FILE* fp = fopen("results.json", "w");
    fprintf(fp, "{\n");
    fprintf(fp, "  \"total_cards\": %d,\n", NUM_CARDS);
    fprintf(fp, "  \"valid_cards\": %d,\n", valid_count);
    fprintf(fp, "  \"invalid_cards\": %d,\n", invalid_count);
    fprintf(fp, "  \"valid_percentage\": %.2f,\n", valid_percentage);
    fprintf(fp, "  \"generation_time\": %.4f,\n", gen_time);
    fprintf(fp, "  \"validation_time\": %.4f,\n", val_time);
    fprintf(fp, "  \"cuda_kernel_time\": %.4f,\n", cuda_time);
    fprintf(fp, "  \"total_time\": %.4f,\n", total_time);
    fprintf(fp, "  \"throughput\": %.0f,\n", throughput);
    fprintf(fp, "  \"cpu_threads\": %d\n", omp_get_max_threads());
    fprintf(fp, "}\n");
    fclose(fp);

    // Save sample cards (first 1000)
    int* all_results = (int*)malloc(NUM_CARDS * sizeof(int));

    // Re-validate to get individual results for sampling
    CreditCard* d_cards;
    int* d_results;
    cudaMalloc(&d_cards, NUM_CARDS * sizeof(CreditCard));
    cudaMalloc(&d_results, NUM_CARDS * sizeof(int));
    cudaMemcpy(d_cards, cards, NUM_CARDS * sizeof(CreditCard), cudaMemcpyHostToDevice);

    int threads_per_block = 512;
    int num_blocks = (NUM_CARDS + threads_per_block - 1) / threads_per_block;
    validate_cards_kernel<<<num_blocks, threads_per_block>>>(d_cards, d_results, NUM_CARDS);
    cudaDeviceSynchronize();

    cudaMemcpy(all_results, d_results, NUM_CARDS * sizeof(int), cudaMemcpyDeviceToHost);
    save_sample_cards(cards, all_results, NUM_CARDS, 1000);

    cudaFree(d_cards);
    cudaFree(d_results);
    free(all_results);

    // Display summary
    printf("\n");
    printf("╔═══════════════════════════════════════════════╗\n");
    printf("║       CREDIT CARD VALIDATION SUMMARY          ║\n");
    printf("╚═══════════════════════════════════════════════╝\n");
    printf("\n");
    printf("📊 Dataset Statistics:\n");
    printf("   • Total Cards: %s\n", "10,000,000");
    printf("   • Valid Cards: %s (%.2f%%)\n",
           valid_count >= 1000000 ? "~1M" :
           valid_count >= 100000 ? "~100K" :
           valid_count >= 10000 ? "~10K" : "Few",
           valid_percentage);
    printf("   • Invalid Cards: %s\n",
           invalid_count >= 1000000 ? "~9M" : "Majority");
    printf("\n");
    printf("⏱ Performance Metrics:\n");
    printf("   • Generation Time: %.4f sec\n", gen_time);
    printf("   • Validation Time: %.4f sec\n", val_time);
    printf("   • CUDA Kernel Time: %.4f sec\n", cuda_time);
    printf("   • Total Execution: %.4f sec\n", total_time);
    printf("\n");
    printf("Throughput:\n");
    printf("   • Speed: %.2f million cards/sec\n", throughput/1000000.0);
    printf("\n");
    printf("System Configuration:\n");
    printf("   • GPU: Tesla T4 (2,560 CUDA cores)\n");
    printf("   • CPU Threads: %d (OpenMP)\n", omp_get_max_threads());
    printf("   • Architecture: CUDA + OpenMP Hybrid\n");
    printf("\n");
    printf("Results saved to: results.json\n");
    printf("Sample cards saved to: sample_cards.json\n");
    printf("\n");

    free(cards);
    return 0;
}

Appending to pdc-project-sp22-bcs-010-and-032.cu


## **Compilation**

In [39]:
print("Compiling CUDA code...")
!nvcc -arch=sm_75 -O2 -Xcompiler -fopenmp pdc-project-sp22-bcs-010-and-032.cu -o pdc-project-sp22-bcs-010-and-032
print("Compilation successful!\n")

Compiling CUDA code...
Compilation successful!



## **Execution**

In [40]:
print("Running Credit Card Validator...")
!./pdc-project-sp22-bcs-010-and-032

Running Credit Card Validator...
Generating 10000000 credit cards using OpenMP...
Generation complete: 6.7581 seconds

Validating cards using CUDA + OpenMP...
Validation complete: 0.2983 seconds

💾 Saving results to JSON...

╔═══════════════════════════════════════════════╗
║       CREDIT CARD VALIDATION SUMMARY          ║
╚═══════════════════════════════════════════════╝

📊 Dataset Statistics:
   • Total Cards: 10,000,000
   • Valid Cards: ~100K (10.00%)
   • Invalid Cards: ~9M

⏱ Performance Metrics:
   • Generation Time: 6.7581 sec
   • Validation Time: 0.2983 sec
   • CUDA Kernel Time: 0.0041 sec
   • Total Execution: 7.0564 sec

Throughput:
   • Speed: 1.42 million cards/sec

System Configuration:
   • GPU: Tesla T4 (2,560 CUDA cores)
   • CPU Threads: 2 (OpenMP)
   • Architecture: CUDA + OpenMP Hybrid

Results saved to: results.json
Sample cards saved to: sample_cards.json



## **Loading Results & sample cards for reference**

In [41]:
import json

# Load results
with open('results.json', 'r') as f:
    results = json.load(f)

# Load sample cards
with open('sample_cards.json', 'r') as f:
    sample_cards = json.load(f)

print(f"\nLoaded {len(sample_cards)} sample cards")
print(f"Results: {results['valid_cards']:,} valid out of {results['total_cards']:,}")


Loaded 1000 sample cards
Results: 999,777 valid out of 10,000,000


## **Dashboard Creation**

In [42]:
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content-width=device-width, initial-scale=1.0">
    <title>CREDIT CARD PATTERN MATCHING</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        body {{ font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif; }}
        .tab-content {{ display: none; }}
        .tab-content.active {{ display: block; }}
        .animate-fade-in {{ animation: fadeIn 0.5s; }}
        @keyframes fadeIn {{ from {{ opacity: 0; }} to {{ opacity: 1; }} }}

        /* Custom scrollbar for dark theme */
        ::-webkit-scrollbar {{ width: 10px; height: 10px; }}
        ::-webkit-scrollbar-track {{ background: #1f2937; }}
        ::-webkit-scrollbar-thumb {{ background: #4b5563; border-radius: 5px; }}
        ::-webkit-scrollbar-thumb:hover {{ background: #6b7280; }}
    </style>
</head>
<body class="bg-gray-900 min-h-screen">
    <div class="max-w-7xl mx-auto p-6">
        <!-- Header -->
        <div class="bg-gradient-to-r from-indigo-600 via-purple-600 to-pink-600 rounded-2xl shadow-2xl p-8 mb-6 text-white relative overflow-hidden">
            <div class="absolute inset-0 bg-black opacity-20"></div>
            <div class="relative z-10">
                <h1 class="text-4xl font-bold mb-2">Credit Card Validation System</h1>
                <p class="text-indigo-100 text-lg mb-4">CUDA + OpenMP Hybrid Parallel Computing</p>
                <div class="flex flex-wrap gap-4 text-sm">
                    <div class="flex items-center gap-2 bg-white bg-opacity-20 px-4 py-2 rounded-lg backdrop-blur-sm">
                        <span>🖥️ Tesla T4 GPU</span>
                    </div>
                    <div class="flex items-center gap-2 bg-white bg-opacity-20 px-4 py-2 rounded-lg backdrop-blur-sm">
                        <span>⚡ {results['total_cards']:,} Cards</span>
                    </div>
                    <div class="flex items-center gap-2 bg-white bg-opacity-20 px-4 py-2 rounded-lg backdrop-blur-sm">
                        <span>📈 {results['throughput']/1000000:.2f}M cards/sec</span>
                    </div>
                </div>
            </div>
        </div>

        <!-- Tabs -->
        <div class="bg-gray-800 rounded-xl shadow-xl mb-6 border border-gray-700">
            <div class="flex border-b border-gray-700">
                <button onclick="showTab('dashboard')" class="tab-btn flex-1 px-6 py-4 font-semibold border-b-2 border-indigo-500 text-indigo-400 transition-all">
                    📊 Dashboard
                </button>
                <button onclick="showTab('cards')" class="tab-btn flex-1 px-6 py-4 font-semibold text-gray-400 hover:text-indigo-400 transition-all">
                    💳 Generated Cards
                </button>
                <button onclick="showTab('source')" class="tab-btn flex-1 px-6 py-4 font-semibold text-gray-400 hover:text-indigo-400 transition-all">
                    💻 Source Code
                </button>
            </div>
        </div>

        <!-- Dashboard Tab -->
        <div id="dashboard" class="tab-content active animate-fade-in">
            <!-- Key Metrics -->
            <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4 mb-6">
                <div class="bg-gradient-to-br from-blue-600 to-blue-700 rounded-xl p-6 text-white shadow-xl border border-blue-500 border-opacity-20">
                    <p class="text-blue-200 text-sm font-medium">Total Cards</p>
                    <p class="text-4xl font-bold mt-2">{results['total_cards']/1000000:.1f}M</p>
                </div>
                <div class="bg-gradient-to-br from-emerald-600 to-emerald-700 rounded-xl p-6 text-white shadow-xl border border-emerald-500 border-opacity-20">
                    <p class="text-emerald-200 text-sm font-medium">Valid Cards</p>
                    <p class="text-4xl font-bold mt-2">{results['valid_cards']/1000:.0f}K</p>
                    <p class="text-emerald-200 text-xs mt-1">{results['valid_percentage']:.2f}%</p>
                </div>
                <div class="bg-gradient-to-br from-orange-600 to-orange-700 rounded-xl p-6 text-white shadow-xl border border-orange-500 border-opacity-20">
                    <p class="text-orange-200 text-sm font-medium">Throughput</p>
                    <p class="text-4xl font-bold mt-2">{results['throughput']/1000000:.2f}M</p>
                    <p class="text-orange-200 text-xs mt-1">cards/sec</p>
                </div>
                <div class="bg-gradient-to-br from-purple-600 to-purple-700 rounded-xl p-6 text-white shadow-xl border border-purple-500 border-opacity-20">
                    <p class="text-purple-200 text-sm font-medium">Total Time</p>
                    <p class="text-4xl font-bold mt-2">{results['total_time']:.2f}s</p>
                </div>
            </div>

            <!-- Charts -->
            <div class="grid grid-cols-1 lg:grid-cols-2 gap-6 mb-6">
                <div class="bg-gray-800 rounded-xl shadow-xl p-6 border border-gray-700">
                    <h3 class="text-lg font-bold mb-4 text-gray-100">Validation Results</h3>
                    <canvas id="pieChart"></canvas>
                </div>
                <div class="bg-gray-800 rounded-xl shadow-xl p-6 border border-gray-700">
                    <h3 class="text-lg font-bold mb-4 text-gray-100">Performance Breakdown</h3>
                    <canvas id="barChart"></canvas>
                </div>
            </div>

            <!-- Timeline -->
            <div class="bg-gray-800 rounded-xl shadow-xl p-6 border border-gray-700">
                <h3 class="text-lg font-bold mb-4 text-gray-100">Execution Timeline</h3>
                <div class="space-y-4">
                    <div>
                        <div class="flex justify-between mb-2">
                            <span class="font-medium text-sm text-gray-300">OpenMP Generation</span>
                            <span class="text-sm font-bold text-blue-400">{results['generation_time']:.4f}s</span>
                        </div>
                        <div class="w-full bg-gray-700 rounded-full h-4">
                            <div class="bg-gradient-to-r from-blue-500 to-blue-600 h-4 rounded-full shadow-lg" style="width: {(results['generation_time']/results['total_time'])*100}%"></div>
                        </div>
                    </div>
                    <div>
                        <div class="flex justify-between mb-2">
                            <span class="font-medium text-sm text-gray-300">CUDA Validation</span>
                            <span class="text-sm font-bold text-orange-400">{results['validation_time']:.4f}s</span>
                        </div>
                        <div class="w-full bg-gray-700 rounded-full h-4">
                            <div class="bg-gradient-to-r from-orange-500 to-orange-600 h-4 rounded-full shadow-lg" style="width: {(results['validation_time']/results['total_time'])*100}%"></div>
                        </div>
                    </div>
                </div>
            </div>
        </div>

        <!-- Cards Tab -->
        <div id="cards" class="tab-content">
            <div class="bg-gray-800 rounded-xl shadow-xl p-6 border border-gray-700">
                <div class="flex justify-between items-center mb-6">
                    <h3 class="text-xl font-bold text-gray-100">Generated Credit Cards (Sample: 1,000 of 10M)</h3>
                    <div class="text-sm text-gray-400">
                        Total: <span class="font-bold text-indigo-400">{results['total_cards']:,}</span>
                    </div>
                </div>
                <div class="overflow-auto max-h-[600px] border border-gray-700 rounded-lg">
                    <table class="w-full text-sm">
                        <thead class="bg-gray-900 sticky top-0">
                            <tr>
                                <th class="px-4 py-3 text-left font-semibold text-gray-300">ID</th>
                                <th class="px-4 py-3 text-left font-semibold text-gray-300">Card Number</th>
                                <th class="px-4 py-3 text-left font-semibold text-gray-300">Type</th>
                                <th class="px-4 py-3 text-left font-semibold text-gray-300">Status</th>
                            </tr>
                        </thead>
                        <tbody>
"""

# Add card rows
for card in sample_cards:
    bg_color = "bg-gray-800" if card['id'] % 2 == 0 else "bg-gray-750"
    type_color = "bg-blue-900 text-blue-300 border border-blue-700" if card['type'] == "VISA" else "bg-orange-900 text-orange-300 border border-orange-700"
    status = "✅ Valid" if card['valid'] else "❌ Invalid"
    status_color = "text-emerald-400" if card['valid'] else "text-red-400"

    html_content += f"""
                            <tr class="{bg_color}">
                                <td class="px-4 py-3 font-mono text-gray-400">{card['id']}</td>
                                <td class="px-4 py-3 font-mono text-gray-200">{card['number']}</td>
                                <td class="px-4 py-3">
                                    <span class="px-3 py-1 rounded-full text-xs font-semibold {type_color}">{card['type']}</span>
                                </td>
                                <td class="px-4 py-3 font-semibold {status_color}">{status}</td>
                            </tr>
"""

html_content += f"""
                        </tbody>
                    </table>
                </div>
            </div>
        </div>

        <!-- Source Code Tab -->
        <div id="source" class="tab-content">
            <div class="bg-gray-800 rounded-xl shadow-xl p-6 text-gray-100 border border-gray-700">
                <h3 class="text-xl font-bold mb-4 text-gray-100">CUDA + OpenMP Source Code</h3>
                <div class="overflow-auto max-h-[700px] bg-gray-950 rounded-lg p-4 border border-gray-800">
                    <pre class="text-sm"><code><span class="text-purple-400">// Credit Card Validator - CUDA + OpenMP</span>
<span class="text-pink-400">#include</span> <span class="text-green-400">&lt;stdio.h&gt;</span>
<span class="text-pink-400">#include</span> <span class="text-green-400">&lt;stdlib.h&gt;</span>
<span class="text-pink-400">#include</span> <span class="text-green-400">&lt;omp.h&gt;</span>
<span class="text-pink-400">#include</span> <span class="text-green-400">&lt;cuda_runtime.h&gt;</span>

<span class="text-pink-400">#define</span> <span class="text-blue-400">VISA</span> <span class="text-yellow-400">0</span>
<span class="text-pink-400">#define</span> <span class="text-blue-400">MASTERCARD</span> <span class="text-yellow-400">1</span>
<span class="text-pink-400">#define</span> <span class="text-blue-400">NUM_CARDS</span> <span class="text-yellow-400">10000000</span>

<span class="text-purple-400">// OpenMP Parallel Generation</span>
<span class="text-pink-400">void</span> <span class="text-yellow-400">initialize_cards_parallel</span>(<span class="text-blue-300">CreditCard*</span> cards, <span class="text-pink-400">int</span> count) {{
    <span class="text-pink-400">#pragma</span> <span class="text-blue-400">omp parallel for</span>
    <span class="text-pink-400">for</span> (<span class="text-pink-400">int</span> i = <span class="text-yellow-400">0</span>; i &lt; count; i++) {{
        <span class="text-purple-400">// Generate cards in parallel</span>
    }}
}}

<span class="text-purple-400">// CUDA GPU Validation Kernel</span>
<span class="text-blue-400">__global__</span> <span class="text-pink-400">void</span> <span class="text-yellow-400">validate_cards_kernel</span>(<span class="text-blue-300">CreditCard*</span> cards,
                                      <span class="text-pink-400">int*</span> results, <span class="text-pink-400">int</span> count) {{
    <span class="text-pink-400">int</span> idx = blockIdx.x * blockDim.x + threadIdx.x;
    <span class="text-pink-400">if</span> (idx &lt; count) {{
        <span class="text-purple-400">// Validate each card on GPU</span>
        results[idx] = check_pattern() && luhn_check();
    }}
}}

<span class="text-purple-400">// Main Hybrid Pipeline</span>
<span class="text-pink-400">int</span> <span class="text-yellow-400">main</span>() {{
    <span class="text-purple-400">// Phase 1: OpenMP Generation</span>
    initialize_cards_parallel(cards, NUM_CARDS);

    <span class="text-purple-400">// Phase 2: CUDA Validation</span>
    validate_cards_kernel&lt;&lt;&lt;blocks, threads&gt;&gt;&gt;(d_cards, d_results, NUM_CARDS);

    <span class="text-purple-400">// Phase 3: OpenMP Aggregation</span>
    count_valid_cards_parallel(results, NUM_CARDS);
}}</code></pre>
                </div>
                <div class="mt-6 bg-gradient-to-r from-indigo-900 to-purple-900 rounded-lg p-4 border border-indigo-700">
                    <h4 class="font-bold text-white mb-2">Key Features:</h4>
                    <ul class="space-y-2 text-sm text-gray-200">
                        <li>✅ OpenMP: Multi-threaded card generation</li>
                        <li>✅ CUDA: Massively parallel GPU validation</li>
                        <li>✅ Luhn Algorithm: Industry-standard checksum</li>
                        <li>✅ Pattern Matching: VISA & MasterCard validation</li>
                        <li>✅ Hybrid: Optimal CPU-GPU workload distribution</li>
                    </ul>
                </div>
            </div>
        </div>

        <!-- Footer -->
        <div class="mt-8 text-center text-gray-500 text-sm">
            <p>PDC Project SP22 • BCS-010 & 032 • Processed {results['total_cards']:,} cards in {results['total_time']:.2f} seconds</p>
        </div>
    </div>

    <script>
        // Tab switching
        function showTab(tabName) {{
            document.querySelectorAll('.tab-content').forEach(tab => {{
                tab.classList.remove('active');
            }});
            document.querySelectorAll('.tab-btn').forEach(btn => {{
                btn.classList.remove('border-indigo-500', 'text-indigo-400');
                btn.classList.add('text-gray-400');
            }});

            document.getElementById(tabName).classList.add('active');
            event.target.classList.add('border-indigo-500', 'text-indigo-400');
            event.target.classList.remove('text-gray-400');
        }}

        // Chart.js dark theme configuration
        Chart.defaults.color = '#9ca3af';
        Chart.defaults.borderColor = '#374151';

        // Pie Chart
        const pieCtx = document.getElementById('pieChart').getContext('2d');
        new Chart(pieCtx, {{
            type: 'doughnut',
            data: {{
                labels: ['Valid Cards', 'Invalid Cards'],
                datasets: [{{
                    data: [{results['valid_cards']}, {results['invalid_cards']}],
                    backgroundColor: ['#10b981', '#ef4444'],
                    borderColor: ['#059669', '#dc2626'],
                    borderWidth: 2
                }}]
            }},
            options: {{
                responsive: true,
                plugins: {{
                    legend: {{
                        position: 'bottom',
                        labels: {{
                            color: '#e5e7eb',
                            font: {{ size: 13 }},
                            padding: 15
                        }}
                    }}
                }}
            }}
        }});

        // Bar Chart
        const barCtx = document.getElementById('barChart').getContext('2d');
        new Chart(barCtx, {{
            type: 'bar',
            data: {{
                labels: ['Generation', 'Validation', 'Kernel Only'],
                datasets: [{{
                    label: 'Time (seconds)',
                    data: [{results['generation_time']}, {results['validation_time']}, {results['cuda_kernel_time']}],
                    backgroundColor: ['#3b82f6', '#f59e0b', '#ef4444'],
                    borderColor: ['#2563eb', '#d97706', '#dc2626'],
                    borderWidth: 2
                }}]
            }},
            options: {{
                responsive: true,
                plugins: {{
                    legend: {{ display: false }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: true,
                        grid: {{
                            color: '#374151'
                        }},
                        ticks: {{
                            color: '#9ca3af'
                        }}
                    }},
                    x: {{
                        grid: {{
                            display: false
                        }},
                        ticks: {{
                            color: '#9ca3af'
                        }}
                    }}
                }}
            }}
        }});
    </script>
</body>
</html>
"""

# Save HTML
with open('dashboard.html', 'w') as f:
    f.write(html_content)

print("Dashboard created: dashboard.html")

Dashboard created: dashboard.html


## **Method to Display the Dashboard**

In [43]:
from IPython.display import HTML, display
import base64

# Read the HTML file
with open('dashboard.html', 'r') as f:
    html_code = f.read()



# **Project Presentation by proven Results**

In [44]:
display(HTML(html_code))

ID,Card Number,Type,Status
1,4936209847528656,VISA,❌ Invalid
2,4735166546826011,VISA,❌ Invalid
3,4235077183135991,VISA,❌ Invalid
4,5364293847622445,MASTERCARD,❌ Invalid
5,5288388006319920,MASTERCARD,❌ Invalid
6,5118637932926161,MASTERCARD,❌ Invalid
7,5229750572115229,MASTERCARD,❌ Invalid
8,4420205128512249,VISA,❌ Invalid
9,4503479626859391,VISA,✅ Valid
10,4235476990041285,VISA,❌ Invalid
